In [1]:
from oemetadata.latest.example import OEMETADATA_LATEST_EXAMPLE
from oemetadata.latest.schema import OEMETADATA_LATEST_SCHEMA
from oemetadata.latest.template import OEMETADATA_LATEST_TEMPLATE

import yaml
from jsonschema import ValidationError, validate
import json
from datetime import date
from pprint import pprint

print(OEMETADATA_LATEST_TEMPLATE)

def check_schema(package):
    try:
        validate(package, OEMETADATA_LATEST_SCHEMA)
        print("Metadata is valid according to OEMetadata Schema (Latest).")
    except ValidationError as e:
        print("Cannot validate the metadata according to OEMetadata Schema (Latest)!", e)

check_schema(OEMETADATA_LATEST_TEMPLATE)

config_file = "../config/public/public_communities_monthly_generation.yml"
metadata_file = "../metadata/public/public_communities_monthly_generation.json"


{'@context': '', 'name': '', 'title': '', 'description': '', 'id': '', 'resources': [{'@id': '', 'name': '', 'topics': [''], 'title': '', 'path': '', 'description': '', 'languages': [''], 'subject': [{'name': '', '@id': ''}], 'keywords': [''], 'publicationDate': '', 'embargoPeriod': {'start': '', 'end': '', 'isActive': False}, 'context': {'title': '', 'homepage': '', 'documentation': '', 'sourceCode': '', 'publisher': '', 'publisherLogo': '', 'contact': '', 'fundingAgency': '', 'fundingAgencyLogo': '', 'grantNo': ''}, 'spatial': {'location': {'address': '', '@id': '', 'latitude': '', 'longitude': ''}, 'extent': {'name': '', '@id': '', 'resolutionValue': '', 'resolutionUnit': '', 'boundingBox': [0, 0, 0, 0], 'crs': ''}}, 'temporal': {'referenceDate': '', 'timeseries': [{'start': '', 'end': '', 'resolutionValue': '', 'resolutionUnit': '', 'alignment': '', 'aggregationType': ''}]}, 'sources': [{'title': '', 'authors': [''], 'description': '', 'publicationYear': '', 'path': '', 'sourceLice

In [2]:
# Read YAML files
with open(config_file, 'r') as stream:
    config = yaml.safe_load(stream)

with open("../registry/fields.yml", 'r') as stream:
    fields = yaml.safe_load(stream)

with open("../registry/licenses.yml", 'r') as stream:
    licenses = yaml.safe_load(stream)

with open("../registry/agents.yml", 'r') as stream:
    agents = yaml.safe_load(stream)

print(fields)

{'fields': {'idxcols': {'fips_code': {'description': 'Federal Information Processing Standard, identifying geographic locations', 'type': 'string', 'nullable': False, 'unit': None}, 'grid_id': {'description': 'A unique identifier of the grid in Alaska as determined for AEDG', 'type': 'string', 'nullable': False, 'unit': None}}, 'cols': {'name': {'description': 'Name', 'type': 'string', 'nullable': False, 'unit': None}, 'net_generation_mwh': {'description': 'Net electrical generation aggregated by grid', 'type': 'number', 'nullable': True, 'unit': 'Megawatt Hour'}, 'fuel_type': {'description': 'Code indicating generation fuel', 'type': 'string', 'nullable': True, 'unit': None}, 'year': {'description': 'Year', 'type': 'integer', 'nullable': False, 'unit': None}, 'month': {'description': 'Month', 'type': 'integer', 'nullable': False, 'unit': None}}}}


In [3]:
# Make some basic changes that will be true of all AEDG metadata
new_pkg = OEMETADATA_LATEST_TEMPLATE.copy()

# None are at a single spatial location
new_pkg['resources'][0]['spatial'].pop('location', None)
# None are embargoed
new_pkg['resources'][0].pop('embargoPeriod', None)
# everything is in US english
new_pkg['resources'][0]['languages'] = ['en-US']
# publishing today
new_pkg['resources'][0]['publicationDate'] = f'{date.today()}'
# we aren't using the OEMetadata review system
new_pkg['resources'][0].pop('review', None)
# CSV files will always be comma delimite
new_pkg['resources'][0]['dialect']['delimiter'] = ","
# We use "." in our floating point numbers
new_pkg['resources'][0]['dialect']['decimalSeparator'] = "."
# ACEP is a contributor to all (fill in details later)
new_pkg['resources'][0]['contributors'] = [
    {'path': 'https://github.com/acep-aedg/aedg-etl-2024',
     'organization': 'Alaska Center for Energy and Power, University of Alaska Fairbanks',
     'date': f'{date.today()}',
     'object': '[Fill in object of the change]',
     'comment': '[Fill in how it was changed]'}
]
# Context is AEDG
new_pkg['resources'][0]['context'] = \
    {'title': 'Alaska Energy Data Gateway',
     'homepage': 'https://akenergygateway.alaska.edu/',
     'publisher': 'Alaska Center for Energy and Power, University of Alaska Fairbanks',
     'fundingAgency': 'State of Alaska'}


# One day we will do Ontology, but this is not the day :(
new_pkg['resources'][0].pop('subject', None)
new_pkg['resources'][0]['schema']['fields'][0].pop('isAbout', None)
new_pkg['resources'][0]['schema']['fields'][0].pop('valueReference', None)


print(new_pkg)
check_schema(new_pkg)

{'@context': '', 'name': '', 'title': '', 'description': '', 'id': '', 'resources': [{'@id': '', 'name': '', 'topics': [''], 'title': '', 'path': '', 'description': '', 'languages': ['en-US'], 'keywords': [''], 'publicationDate': '2025-03-07', 'context': {'title': 'Alaska Energy Data Gateway', 'homepage': 'https://akenergygateway.alaska.edu/', 'publisher': 'Alaska Center for Energy and Power, University of Alaska Fairbanks', 'fundingAgency': 'State of Alaska'}, 'spatial': {'extent': {'name': '', '@id': '', 'resolutionValue': '', 'resolutionUnit': '', 'boundingBox': [0, 0, 0, 0], 'crs': ''}}, 'temporal': {'referenceDate': '', 'timeseries': [{'start': '', 'end': '', 'resolutionValue': '', 'resolutionUnit': '', 'alignment': '', 'aggregationType': ''}]}, 'sources': [{'title': '', 'authors': [''], 'description': '', 'publicationYear': '', 'path': '', 'sourceLicenses': [{'name': '', 'title': '', 'path': '', 'instruction': '', 'attribution': '', 'copyrightStatement': ''}]}], 'licenses': [{'na

In [4]:
# Now copy in configs specific to this file

new_pkg['name'] = config['metadata']['name']
new_pkg['title'] = config['metadata']['title']
new_pkg['description'] = config['metadata']['description']

# resource is same as the package, I guess
new_pkg['resources'][0]['name'] = config['metadata']['name']
new_pkg['resources'][0]['title'] = config['metadata']['title']
new_pkg['resources'][0]['description'] = config['metadata']['description']
new_pkg['resources'][0]['keywords'] = config['metadata']['resources'][0]['keywords']
new_pkg['resources'][0]['topics'] = config['metadata']['resources'][0]['topics']
new_pkg['resources'][0]['path'] = config['metadata']['resources'][0]['path']
if config['metadata']['resources'][0]['path'].endswith(".csv"):
    new_pkg['resources'][0]['type'] = "table"
    new_pkg['resources'][0]['format'] = "CSV"
if config['metadata']['resources'][0]['path'].endswith(".geojson"):  # I don't know if OEMetadata does this
    new_pkg['resources'][0]['type'] = "geospatial"
    new_pkg['resources'][0]['format'] = "GEOJOSN"
    
print(new_pkg)
check_schema(new_pkg)

{'@context': '', 'name': 'public_communities_monthly_generation', 'title': 'Monthly electrical generation by community', 'description': 'Monthly electrical generation is aggregated by grid and associated with every community connected to the grid.\n', 'id': '', 'resources': [{'@id': '', 'name': 'public_communities_monthly_generation', 'topics': ['Electricity'], 'title': 'Monthly electrical generation by community', 'path': 'https://github.com/acep-aedg/aedg-data-pond/blob/main/data/public/public_communities_monthly_generation.csv', 'description': 'Monthly electrical generation is aggregated by grid and associated with every community connected to the grid.\n', 'languages': ['en-US'], 'keywords': ['generation', 'foo', 'bar'], 'publicationDate': '2025-03-07', 'context': {'title': 'Alaska Energy Data Gateway', 'homepage': 'https://akenergygateway.alaska.edu/', 'publisher': 'Alaska Center for Energy and Power, University of Alaska Fairbanks', 'fundingAgency': 'State of Alaska'}, 'spatial':

In [5]:
# Add the license

all_licenses = []
for license_tag in config['metadata']['resources'][0]['licenses']:
    license = {'name': license_tag}
    license.update(licenses['licenses'][license_tag])
    all_licenses.append(license)
    
if len(all_licenses) > 0:
    # replace the empty template field
    new_pkg['resources'][0]['licenses'] = all_licenses
else:
    # remove the empty template field
    new_pkg['resources'][0].pop('resources', None)

print(new_pkg)
check_schema(new_pkg)

{'@context': '', 'name': 'public_communities_monthly_generation', 'title': 'Monthly electrical generation by community', 'description': 'Monthly electrical generation is aggregated by grid and associated with every community connected to the grid.\n', 'id': '', 'resources': [{'@id': '', 'name': 'public_communities_monthly_generation', 'topics': ['Electricity'], 'title': 'Monthly electrical generation by community', 'path': 'https://github.com/acep-aedg/aedg-data-pond/blob/main/data/public/public_communities_monthly_generation.csv', 'description': 'Monthly electrical generation is aggregated by grid and associated with every community connected to the grid.\n', 'languages': ['en-US'], 'keywords': ['generation', 'foo', 'bar'], 'publicationDate': '2025-03-07', 'context': {'title': 'Alaska Energy Data Gateway', 'homepage': 'https://akenergygateway.alaska.edu/', 'publisher': 'Alaska Center for Energy and Power, University of Alaska Fairbanks', 'fundingAgency': 'State of Alaska'}, 'spatial':

In [6]:
# Add the fields

# don't care about the distinction between index and value columns here
all_fields = fields['fields']['idxcols'].copy()
all_fields.update(fields['fields']['cols'])

all_schemas = []
field_names = config['metadata']['resources'][0]['fields']
for field_name in field_names:
    assert field_name in all_fields.keys()
    schema = {'name': field_name}
    schema.update(all_fields[field_name])
    all_schemas.append(schema)
    
assert len(all_schemas) > 0
# replace the empty template field
new_pkg['resources'][0]['schema']['fields'] = all_schemas


print(new_pkg)
check_schema(new_pkg)

{'@context': '', 'name': 'public_communities_monthly_generation', 'title': 'Monthly electrical generation by community', 'description': 'Monthly electrical generation is aggregated by grid and associated with every community connected to the grid.\n', 'id': '', 'resources': [{'@id': '', 'name': 'public_communities_monthly_generation', 'topics': ['Electricity'], 'title': 'Monthly electrical generation by community', 'path': 'https://github.com/acep-aedg/aedg-data-pond/blob/main/data/public/public_communities_monthly_generation.csv', 'description': 'Monthly electrical generation is aggregated by grid and associated with every community connected to the grid.\n', 'languages': ['en-US'], 'keywords': ['generation', 'foo', 'bar'], 'publicationDate': '2025-03-07', 'context': {'title': 'Alaska Energy Data Gateway', 'homepage': 'https://akenergygateway.alaska.edu/', 'publisher': 'Alaska Center for Energy and Power, University of Alaska Fairbanks', 'fundingAgency': 'State of Alaska'}, 'spatial':

In [7]:
pprint(new_pkg, depth=None, sort_dicts=False)

{'@context': '',
 'name': 'public_communities_monthly_generation',
 'title': 'Monthly electrical generation by community',
 'description': 'Monthly electrical generation is aggregated by grid and '
                'associated with every community connected to the grid.\n',
 'id': '',
 'resources': [{'@id': '',
                'name': 'public_communities_monthly_generation',
                'topics': ['Electricity'],
                'title': 'Monthly electrical generation by community',
                'path': 'https://github.com/acep-aedg/aedg-data-pond/blob/main/data/public/public_communities_monthly_generation.csv',
                'description': 'Monthly electrical generation is aggregated by '
                               'grid and associated with every community '
                               'connected to the grid.\n',
                'languages': ['en-US'],
                'keywords': ['generation', 'foo', 'bar'],
                'publicationDate': '2025-03-07',
            

In [8]:
with open(metadata_file, "w") as file:
    json.dump(new_pkg, file, indent=4)